In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
aisles_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/aisles.csv.zip").open("aisles.csv"))
departments_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/departments.csv.zip").open("departments.csv"))
order_products_prior_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/order_products__prior.csv.zip").open("order_products__prior.csv"))
order_products_train_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/order_products__train.csv.zip").open("order_products__train.csv"))
orders_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/orders.csv.zip").open("orders.csv"))
products_df = pd.read_csv(zipfile.ZipFile("../input/instacart-market-basket-analysis/products.csv.zip").open("products.csv"))


In [ ]:
products_df.head()

In [ ]:
order_products_prior_df.info()

In [ ]:
order_products_prior_df.tail()

In [ ]:
orders_df.info()

In [ ]:
orders_df.groupby('eval_set').nunique()

In [ ]:
order_products_train_df.info()

In [ ]:
cnt_srs = orders_df.groupby("user_id")["order_number"].aggregate(np.max).reset_index()
cnt_srs = cnt_srs.order_number.value_counts()

plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Maximum order number', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
grouped_df = orders_df.groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
grouped_df = grouped_df.pivot('order_dow', 'order_hour_of_day', 'order_number')

plt.figure(figsize=(12,6))
sns.heatmap(grouped_df, cmap="YlGnBu",yticklabels=['Sat','Sun', 'Mon','Tue','Wed','Thu','Fri'])
plt.title("Tần suất mua hàng theo ngày trong tuần và giờ trong ngày")
plt.ylabel("Ngày mua hàng")
plt.xlabel("Giờ mua hàng")
plt.show()

Ngày cao điểm mua hàng vào thứ 7(10-16h) và chủ nhật(9-16h).

Ngày thấp điểm mua hàng vào thứ 3 và thứ 4 của tuần

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="days_since_prior_order", data=orders_df)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Days since prior order', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency distribution by days since prior order", fontsize=15)
plt.show()

In [ ]:
#merge table for deeper analysis
order_products_prior_df = pd.merge(order_products_prior_df, products_df, on='product_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, aisles_df, on='aisle_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, departments_df, on='department_id', how='left')
order_products_prior_df = pd.merge(order_products_prior_df, orders_df, on='order_id', how='left')
order_products_prior_df.head()

In [ ]:
cnt_srs = order_products_prior_df['department'].value_counts()
plt.figure(figsize=(12,8))
sns.barplot(cnt_srs.index, cnt_srs.values)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Department', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
temp_series = order_products_prior_df['department'].value_counts()
labels = (np.array(temp_series.index))
sizes = (np.array((temp_series / temp_series.sum())*100))
plt.pie(sizes, labels=labels, 
        autopct='%1.1f%%', startangle=200)
plt.title("Departments distribution", fontsize=15)
plt.show()

In [ ]:
order_products_prior_df['department'].value_counts().index

In [ ]:
for i in order_products_prior_df['department'].value_counts().index:
    print(i)

In [ ]:
order_products_prior_df[order_products_prior_df.department=='dairy eggs'].head()

In [ ]:
gr_dept_df = order_products_prior_df[order_products_prior_df.department=='dairy eggs'].groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()

In [ ]:
for i in order_products_prior_df['department'].value_counts().index:
    gr_dept_df = order_products_prior_df[order_products_prior_df.department==i].groupby(["order_dow", "order_hour_of_day"])["order_number"].aggregate("count").reset_index()
    gr_dept_df = gr_dept_df.pivot('order_dow', 'order_hour_of_day', 'order_number')

    plt.figure(figsize=(6,3))
    sns.heatmap(gr_dept_df, cmap="YlGnBu",yticklabels=['Sat','Sun', 'Mon','Tue','Wed','Thu','Fri'])
    plt.title("Tần suất mua hàng Department: "+i)
    plt.ylabel("Ngày mua hàng")
    plt.xlabel("Giờ mua hàng")
    plt.show()

Tần suất mua hàng theo từng ngành hàng (department) có một số điểm khác biệt so với đơn hàng chung gồm:
- Ngành đồ ăn vặt (snacks), nước uống (beverage), đồ ăn sáng (breakfast) cao điểm mua hàng vào sáng chủ nhật.
- Ngành đồ uống có cồn (alcohol) cao điểm mua hàng vào thứ 4 và thứ 5.
- Ngành đồ cho vật nuôi (pets) cao điểm mua hàng vào thứ 6 và thứ 7.

Nhà bán lẻ có thể nghiên cứu sâu hơn lý do để có chiến lược phù hợp (VD:tăng hiển thị/chạy khuyến mãi/...) tận dụng điểm này

In [ ]:
gr_dept_df